In [2]:
from summary.model_summary import model_summary

In [8]:
from summary import model_madd

ImportError: cannot import name 'model_madd'

In [2]:
from networks.denoise.pydl import ResNet_Den

from MMNet_TBPTT import *
from problems import *

from networks.faceid.mobile import MobileFacenet
from networks.faceid.mobile import ArcMarginProduct

In [3]:
from networks.denoise.pydl import ResNet_Den
from MMNet_TBPTT import *
from problems import *
denoiser = ResNet_Den(5, weightnorm=True)

max_iter = 5
mmnet = MMNet(denoiser, max_iter=max_iter)

faceid = MobileFacenet() 

In [8]:
model = faceid
input_size = (3, 112, 96)
summary = model_summary(model, input_size, query_granularity=1)

PReLU is unsupported!
PReLU is unsupported!
PReLU is unsupported!
PReLU is unsupported!
PReLU is unsupported!
PReLU is unsupported!
PReLU is unsupported!
PReLU is unsupported!
PReLU is unsupported!
PReLU is unsupported!
PReLU is unsupported!
PReLU is unsupported!
PReLU is unsupported!
PReLU is unsupported!
PReLU is unsupported!
PReLU is unsupported!
PReLU is unsupported!
PReLU is unsupported!
PReLU is unsupported!
PReLU is unsupported!
PReLU is unsupported!
PReLU is unsupported!
PReLU is unsupported!
PReLU is unsupported!
PReLU is unsupported!
PReLU is unsupported!
PReLU is unsupported!
PReLU is unsupported!
PReLU is unsupported!
PReLU is unsupported!
PReLU is unsupported!
PReLU is unsupported!
PReLU is unsupported!
PReLU is unsupported!
PReLU is unsupported!
PReLU is unsupported!
PReLU is unsupported!
PReLU is unsupported!
PReLU is unsupported!
PReLU is unsupported!
PReLU is unsupported!
PReLU is unsupported!
PReLU is unsupported!
PReLU is unsupported!
PReLU is unsupported!
PReLU is u

In [ ]:
387M MAdd
1M parameters
inference memory(MB) 29.99

In [4]:
from networks.faceid.resnet import resnet_face18, resnet50
# faceid = resnet_face18(use_se = False)
faceid = resnet50()

model = faceid
input_size = (3, 112, 96)
summary = model_summary(model, input_size, query_granularity=1)


               module name   input shape  output shape  parameter quantity inference memory(MB)         MAdd duration percent
0                    conv1     3 112  96    64 112  96                1728               2.62MB   36,470,784            2.72%
1                      bn1    64 112  96    64 112  96                 128               2.62MB    2,752,512            1.62%
2                     relu    64 112  96    64 112  96                   0               2.62MB      688,128            0.15%
3           layer1.0.conv1    64 112  96    64 112  96                4096               2.62MB   87,392,256            3.71%
4             layer1.0.bn1    64 112  96    64 112  96                 128               2.62MB    2,752,512            1.54%
5           layer1.0.conv2    64 112  96    64  56  48               36864               0.66MB  198,008,832            2.71%
6             layer1.0.bn2    64  56  48    64  56  48                 128               0.66MB      688,128          

In [ ]:
total parameters quantity: 19,8 M
total memory: 26.34MB
MAdd: 2976.5 M

In [9]:
model = mmnet
input_size = (3, 112, 96)
summary = model_summary(model, input_size, query_granularity=1)

TypeError: forward() missing 5 required positional arguments: 'xpre', 'mosaic', 'M', 'noise_sigma', and 'k'

In [6]:
from model_summary import model_summary

In [ ]:
2

In [ ]:
denoiser.conv1.numel()

In [33]:
denoiser = ResNet_Den(5, weightnorm=True)

In [34]:
model = denoiser
input_size = (3, 112, 96)
summary = model_summary(model, input_size, query_granularity=1)

L2Proj is unsupported!
       module name  input shape output shape  parameter quantity inference memory(MB)         MAdd duration percent
0            conv1    3 116 100   64 112  96                4928               2.62MB  103,219,200            3.01%
1   layer1.0.conv1   64 114  98   64 112  96               36992               2.62MB  792,723,456            6.17%
2   layer1.0.relu1   64 112  96   64 112  96                  64               2.62MB    2,752,512            2.85%
3   layer1.0.relu2   64 112  96   64 112  96                  64               2.62MB    2,752,512            2.94%
4   layer1.0.conv2   64 114  98   64 112  96               36992               2.62MB  792,723,456            6.07%
5   layer1.1.conv1   64 114  98   64 112  96               36992               2.62MB  792,723,456            6.11%
6   layer1.1.relu1   64 112  96   64 112  96                  64               2.62MB    2,752,512            2.79%
7   layer1.1.relu2   64 112  96   64 112  96     

In [32]:
import sys
sys.path.append("/home/safin/")

import torch.nn as nn
import torch as th

from pydl.networks.UDNet.net import UDNet
from pydl.networks.ResDNet.net import ResDNet

from pydl.nnLayers import modules
from pydl.nnLayers import init
from pydl.utils import loadmat
from pydl.nnLayers.cascades import nconv2D, nconv_transpose2D
from pydl.utils import formatInput2Tuple, getPad2RetainShape
import torch.nn.functional as F

import torch
from torch.nn.utils import weight_norm

import torch.nn.functional as F
import torch.nn.init as init
    
class L2Proj(nn.Module):
    # L2Prox layer
    # Y = NN_L2TRPROX(X,EPSILON) computes the Proximal map layer for the
    #   indicator function :
    #
    #                      { 0 if ||X|| <= EPSILON
    #   i_C(D,EPSILON){X}= {
    #                      { +inf if ||X|| > EPSILON
    #
    #   X and Y are of size H x W x K x N, and EPSILON = exp(ALPHA)*V*STDN
    #   is a scalar or a 1 x N vector, where V = sqrt(H*W*K-1).
    #
    #   Y = K*X where K = EPSILON / max(||X||,EPSILON);
    # s.lefkimmiatis@skoltech.ru, 22/11/2016.
    # pytorch implementation filippos.kokkinos@skoltech.ru 1/11/2017

    def __init__(self):
        super(L2Proj, self).__init__()

    def forward(self, x, stdn, alpha):
        if x.is_cuda:
            x_size = torch.cuda.FloatTensor(1).fill_(x.shape[1] * x.shape[2] * x.shape[3])
        else:
            x_size = torch.Tensor([x.shape[1] * x.shape[2] * x.shape[3]])
        numX = torch.sqrt(x_size-1)
        if x.is_cuda:
            epsilon = torch.cuda.FloatTensor(x.shape[0],1,1,1).fill_(1) * (torch.exp(alpha) * stdn * numX).view(-1,1,1,1)
        else:
            epsilon = torch.zeros(x.size(0),1,1,1).fill_(1) * (torch.exp(alpha) *  stdn * numX).view(-1,1,1,1)
        x_resized = x.view(x.shape[0], -1)
        x_norm = torch.norm(x_resized, 2, dim=1).view(x.size(0),1,1,1)
        max_norm = torch.max(x_norm, epsilon)
        result = x * (epsilon / max_norm)
        return result

def conv3x3(in_planes, out_planes, stride=1):
    "3x3 convolution with padding"
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride,
                     padding=0, bias=True)

class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, inplanes, planes, stride=1, weightnorm=None, shortcut=True):
        super(BasicBlock, self).__init__()
        self.shortcut = shortcut
        self.conv1 = conv3x3(inplanes, planes, stride)

        self.relu1 = nn.PReLU(num_parameters=planes,init=0.1)
        self.relu2 = nn.PReLU(num_parameters=planes, init=0.1)
        self.conv2 = conv3x3(inplanes, planes, stride)
        if weightnorm:
            self.conv1 = weight_norm(self.conv1)
            self.conv2 = weight_norm(self.conv2)


    def forward(self, x):
        out = self.relu1(x)
        out = F.pad(out,(1,1,1,1),'reflect')
        out = self.conv1(out)
        out = out[:,:, :x.shape[2], :x.shape[3]]
        out = self.relu2(out)
        out = F.pad(out,(1,1,1,1),'reflect')
        out = self.conv2(out)
        out = out[:,:, :x.shape[2], :x.shape[3]]
        if self.shortcut:
            out = x + out
        return out

import numpy as np


class ResNet_Den(nn.Module):

    def __init__(self, layer_size, color=True, weightnorm=None):
        self.inplanes = 64
        block = BasicBlock
        super(ResNet_Den, self).__init__()
        if color:
            in_channels = 3
        else:
            in_channels = 1

        self.conv1 = nn.Conv2d(in_channels, 64, kernel_size=5, stride=1, padding=0,
                               bias=True)
        if weightnorm:
            self.conv1 = weight_norm(self.conv1)

        # inntermediate layer has D-2 depth
        self.layer1 = self._make_layer(block, 64, layer_size)
        self.conv_out = nn.ConvTranspose2d(64, in_channels, kernel_size=5, stride=1, padding=2,
                                  bias=True)
        if weightnorm:
            self.conv_out = weight_norm(self.conv_out)

        self.l2proj = L2Proj()

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                weights = np.sqrt(2/(9.*64))*np.random.standard_normal(m.weight.data.shape)
                #weights = np.random.normal(size=m.weight.data.shape,
                #                           scale=np.sqrt(1. / m.weight.data.shape[1]))
                m.weight.data = torch.Tensor(weights)
                if m.bias is not None:
                    m.bias.data.zero_()
        self.zeromean()

    def _make_layer(self, block, planes, blocks, stride=1):
        layers = []
        layers.append(block(self.inplanes, planes, stride, weightnorm=True, shortcut=False))

        for i in range(1, blocks):
            layers.append(block(self.inplanes, planes, weightnorm=True, shortcut=True))
        return nn.Sequential(*layers)

    def zeromean(self):
        # Function zeromean subtracts the mean E(f) from filters f
        # in order to create zero mean filters
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                m.weight.data = m.weight.data - torch.mean(m.weight.data)

    def forward(self, x):
        self.zeromean()
        out = F.pad(x,(2,2,2,2),'reflect')
        out = self.conv1(out)
        out = self.layer1(out)
        out = self.conv_out(out)
        stdn = torch.FloatTensor(1).fill_(1)
        sigma_min = 1
        sigma_max = 2
        alpha = nn.Parameter(torch.Tensor(1))
        out = self.l2proj(out, stdn, alpha)
        return out


In [31]:
del ResNet_Den

In [3]:
from networks.denoise.dncnn import DnCNN
dncnn = DnCNN(image_channels=3)

In [4]:
model = dncnn
input_size = (3, 112, 96)
summary = model_summary(model, input_size, query_granularity=1)

   module name  input shape output shape  parameter quantity inference memory(MB)         MAdd duration percent
0      dncnn.0    3 112  96   64 112  96                1792               2.62MB   37,158,912            1.98%
1      dncnn.1   64 112  96   64 112  96                   0               2.62MB      688,128           37.08%
2      dncnn.2   64 112  96   64 112  96               36864               2.62MB  792,035,328            2.41%
3      dncnn.3   64 112  96   64 112  96                 128               2.62MB    2,752,512            3.82%
4      dncnn.4   64 112  96   64 112  96                   0               2.62MB      688,128            0.07%
5      dncnn.5   64 112  96   64 112  96               36864               2.62MB  792,035,328            2.83%
6      dncnn.6   64 112  96   64 112  96                 128               2.62MB    2,752,512            0.42%
7      dncnn.7   64 112  96   64 112  96                   0               2.62MB      688,128          

In [1]:
import time
import torch
from networks.denoise.dncnn import DnCNN
from utils import freeze_model
dncnn = DnCNN(image_channels=3)

model = dncnn.cuda()
model = model.eval()
model = model.train(False)
freeze_model(model)
input_size = (128, 3, 112, 96)
input_tnsr = torch.randn(input_size).cuda()
n = 10
times = []
for i in range(10):
    s = time.time()
    for i in range(n):
        model(input_tnsr)
    f = time.time()
    times.append(f-s)

In [2]:
import numpy as np
np.mean(times)/10 * 1000

138.69744062423706

3.478386402130127

In [4]:
np.std(times)/10 * 1000

5.339246084530411

In [1]:
def generate_mask(im_shape, pattern='RGGB'):
    if pattern == 'RGGB':
        # pattern RGGB
        r_mask = torch.zeros(im_shape)
        r_mask[0::2, 0::2] = 1

        g_mask = torch.zeros(im_shape)
        g_mask[::2, 1::2] = 1
        g_mask[1::2, ::2] = 1

        b_mask = torch.zeros(im_shape)
        b_mask[1::2, 1::2] = 1

        mask = torch.zeros(im_shape + (3,))
        mask[:, :, 0] = r_mask
        mask[:, :, 1] = g_mask
        mask[:, :, 2] = b_mask
        
        return mask

In [2]:
from networks.denoise.pydl import ResNet_Den
from MMNet_TBPTT import *
from problems import *
from utils import freeze_model
denoiser = ResNet_Den(5, weightnorm=True).cuda()
denoiser = denoiser.eval()
denoiser = denoiser.train(False)
freeze_model(denoiser)

max_iter = 5
mmnet = MMNet(denoiser, max_iter=max_iter).cuda()
mmnet = mmnet.eval()
mmnet = mmnet.train(False)
freeze_model(mmnet)

In [3]:
model = mmnet

input_size = (16, 3, 112, 96)
input_tnsr = torch.randn(input_size).cuda()
M = generate_mask((112, 96), pattern='RGGB').permute(2,0,1).unsqueeze(0).cuda()
n = 10
times = []
for i in range(10):
    s = time.time()
    for i in range(n):
        model.forward_all_iter(input_tnsr, M, init=False, noise_estimation=True)
    f = time.time()
    times.append(f-s)

In [4]:
import numpy as np
np.mean(times)/10 * 1000

110.81122398376463